#### Ejercicio de codificación - Compresión de contexto

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Se presenta el esqueleto de la función constitucion que 
procesa un texto y aplica un compresor de contexto al resultado.
Se trabaja con un texto sobre la Constitucion Argentina tomado de Wikipedia. 
La tarea es procesarlo y finalmente aplicar un compresor de contexto.
Los pasos a seguir están indicados.
Se dan las importaciones.
Con --- se indican partes de sentencias que deben completar.

In [2]:
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 

In [3]:
def constitucion(consulta):
    # 1 - Cargar el texto (sentencia ya lista)
    loader = ---Loader("./directorio/ConstitucionArgentina.txt")
    doc = loader.---()
    
    # 2 - Particionar en chunks (varie el tamaño)
    text_splitter = ---.from_tiktoken_encoder(
        chunk_size=---)
    splits = text_splitter.---(doc)
    
    # 3 - Incrustar los chunks y guardar en BD ChromaDB
    embedding = OpenAIEmbeddings()
    db = Chroma.---(documents=splits, 
                               embedding=---,
                               persist_directory="./constitucion/"
                              )
                               
    db.persist()

    # 4 - Usar ChatOpenAI y ContextualCompressionRetriever para recuperar los
    #      chunks mas relevantes
    llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0, max_tokens=250)
    compressor = ---.from_llm(llm)
    compression_retriever = ---(
        base_compressor=compressor,
        base_retriever=db.---())

# Las siguientes dos sentencias implementan el modo context compression
    compressed_docs = ---.get_relevant_documents(consulta)
    return compressed_docs[0].---

# Invocar la función con:
print(constitucion("Cuando se reformo la constitucion."))

SyntaxError: invalid syntax (451147844.py, line 4)

##### Solución

In [5]:
def constitucion(consulta):
    # 1 - Cargar el texto (sentencia ya lista)
    loader = TextLoader("./directorio/ConstitucionArgentina.txt")
    doc = loader.load()
    # 2 - Particionar en chunks (varie el tamaño)
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000)
    splits = text_splitter.split_documents(doc)
    # 3 - Incrustar los chunks y guardar en BD ChromaDB
    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma.from_documents(documents=splits, 
                               embedding=embedding,
                               persist_directory="./constitucion/"
                              )
                               
    db.persist()
    # 4 - Usar ChatOpenAI y ContextualCompressionRetriever para recuperar los
    #      chunks mas relevantes
    llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0, max_tokens=250)
    compressor = LLMChainExtractor.from_llm(llm)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=db.as_retriever())

# Las siguientes dos sentencias implementan el modo context compression
    compressed_docs = compression_retriever.get_relevant_documents(consulta)
    return compressed_docs[0].page_content
print(constitucion("Cuando se reformo la constitucion."))

c:\nuevochatbot\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


La Constitución de la Nación Argentina, que rige actualmente a la República Argentina, fue aprobada por una asamblea constituyente integrada por representantes de trece provincias, hecha en la ciudad de Santa Fe en el año 1853. El texto ha sido reformado varias veces, estando vigente la redacción resultante de la reforma de 1994... Actualmente, el preámbulo de la Constitución Argentina es el siguiente: Nos los representantes del pueblo de la Nación Argentina, reunidos en Congreso General Constituyente por voluntad y elección de las provincias que la componen, en cumplimiento de pactos preexistentes, con el objeto de constituir la unión nacional...

The Constitution of the Nation Argentina, which currently governs the Republic of Argentina, was approved by an assembly constituent integrated by representatives of thirteen provinces, made in the city of Santa Fe in the year 1853. The text has been reformed several times, with the wording resulting from the reform of
